In [0]:
from pyspark.sql import SparkSession

# Cria a sessão do Spark
spark = SparkSession.builder.appName("AulaSpark").getOrCreate()

In [0]:
# Criação do data set
import numpy as np      # Cria amostra e analises estatisticas
import pandas as pd     # Usado para estruturação

# Cria amostra aleatoria com uma semente
np.random.seed(42)
N = 1000000

nomes = ["Pedro", "Gabrielle", "Bunster", "Kauã", "Ana", 
         "Henrique", "Erica", "Kaio", "Fernanda", "Isabella"]

paises = ["Brasil", "EUA", "Japão", "Australia"]

data = pd.DataFrame({
    "id": np.arange(1, N+1),
    "nome": np.random.choice(nomes, N),
    "pais": np.random.choice(paises, N),
    "idade": np.random.randint(18, 65, N),
    "salario": np.random.randint(2000, 20000, N),
    "bonus": np.random.randint(0, 2, N)
})

df = spark.createDataFrame(data)
display(df)

In [0]:
# Explorando o dataframe
display(df.limit(10))   # Exibe 10 colunas aleatorias

In [0]:
# Analisa a schema de cada coluna
df.printSchema()

In [0]:
# Descreve as colunas descritas
df.describe(["idade", "salario", "bonus"]).show()

In [0]:
from pyspark.sql.functions import avg, max, min, desc

In [0]:
df.groupBy("pais").agg(
    avg("salario").alias("media_salario"),
    max("salario").alias("max_salario"),
    min("salario").alias("min_salario")
).orderBy(desc("media_salario")).show()

In [0]:
df.orderBy(desc("salario")).show(10)

In [0]:
# Para usar SQL com python - cria a tabela e usa os comandos
df.createOrReplaceTempView("funcionarios")

spark.sql("""
    SELECT pais,
    COUNT(*) AS qtde_func,
    ROUND(AVG(salario), 2) as media_salario,
    ROUND(SUM(salario), 2) as folha_total
    FROM funcionarios
    GROUP BY pais
    ORDER BY folha_total DESC
""").show()

In [0]:
%sql
SELECT COUNT(*) AS qtde_func,
  ROUND(AVG(salario), 2) as media_salario,
  ROUND(SUM(salario), 2) as folha_total
FROM funcionarios
GROUP BY pais
ORDER BY folha_total DESC

In [0]:
# Criação de gráficos (pandas e matplotlib)

import matplotlib.pyplot as plt

df_pd = df.groupBy("pais").agg(avg("salario").alias("media_salario")).toPandas()
df_pd.plot(kind="bar", x="pais", y="media_salario", legend = False)
plt.ylabel("Salario medio")
plt.title("Salario medio por pais")
plt.show()